In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

# Nicer plotting
plt.rcParams["font.weight"] = "bold"
plt.rcParams["font.size"] = "18"
plt.rcParams["axes.labelweight"] = "bold"

# Force only P100 GPU
import os
os.environ['CUDA_VISIBLE_DEVICES']="0"

import torch

# Device configuration
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

### Set up local parameters

In [ ]:
n_epochs = 200
name = 'July_31_75000_2layer'

In [ ]:
# When you type import X.py,
# Python searches sys.path for a python
# file named X.py to import.
#
# Add the directory with the model
# definitions to the path so we can import it
import sys
sys.path.append('../model')

from collectdata import collect_data
from loss import Loss
from training import trainNet
from models import SimpleCNN2Layer as Model

In [ ]:
dataset_train, dataset_val, _ = collect_data(
    "/data/schreihf/PvFinder/July_31_75000.npz", 55_000, 10_000,
    device=device)

In [ ]:
seed = 42
np.random.seed(seed)
torch.manual_seed(seed);

In [ ]:
model = Model().to(device)

In [ ]:
results = trainNet(model, dataset_train, dataset_val, Loss(), 32, n_epochs, learning_rate=1e-3, name=name)
# Add name=name to save each model param file

print()
print("Training finished, took {:.2f}s".format(results.time))

In [ ]:
torch.save(model.state_dict(), f'{name}_final.pyt')

In [ ]:
fig=plt.figure() 
fig.set_figheight(10)
fig.set_figwidth(15)
plt.plot(np.arange(len(results.cost))+1, results.cost, 'o-',color='r',label='Train')
plt.plot(np.arange(len(results.val))+1, results.val, 'o-' , color='b', label='Validation')
plt.xlabel('Number of epoch', weight='bold', size= 20)
plt.ylabel('Average cost per bin of a batch',  weight='bold', size= 20)
plt.yscale('log') 
plt.tick_params('y', colors = 'k',labelsize=16 )
plt.tick_params('x', colors = 'k',labelsize=16 )
plt.legend()
fig.savefig(name + '.png')